In [ ]:
import os
import glob
import gc
import subprocess
import datetime as dt

from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from matplotlib.patches import Patch

import cmocean as cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pathos.multiprocessing import Pool

from scipy.ndimage import zoom, distance_transform_edt, generic_filter, median_filter, gaussian_filter

%matplotlib inline

In [ ]:
src_crs = ccrs.LambertAzimuthalEqualArea(central_longitude=0, central_latitude=90)
mesh_init_file = '../mesh_arctic_ease_25km_max7.npz'
mask = np.load(mesh_init_file)['mask']
xc = np.load(mesh_init_file)['xc']
yc = np.load(mesh_init_file)['yc'][::-1]

In [ ]:
figsize = (10,10)
dpi = 100
dst_dir = './'

xlim = 3500000
ylim = 3500000
plt_csr = ccrs.LambertAzimuthalEqualArea(central_longitude=0, central_latitude=90)

prefix = 'ice_age'
src_dir = '../NERSC_arctic25km_sea_ice_age_v2p1'
src_files = sorted(glob.glob(f'{src_dir}/nc/*/*nc'))[::5]
print(len(src_files), src_files[0], src_files[-1])

clim = [0, 4]
cmap = 'plasma'
var_name = 'age'
def get_date(i, src_file):
    return src_file.split('.')[-2].split('_')[-1]

def get_lmsia_data(ifile, get_land=False):
    with Dataset(ifile) as ds:
        sia = ds.variables['sea_ice_age'][0]
        if get_land:
            status_flag = ds.variables['status_flag'][0]
    if get_land:
        land = np.where(status_flag == 2, 1, np.nan)
    else:
        land = None
    return sia, land

ifile = '../NERSC_arctic25km_sea_ice_age_v2p1/nc/2020/arctic25km_sea_ice_age_v2p1_20200101.nc'
sia, land = get_lmsia_data(ifile, True)

extent = np.array([xc.min(), xc.max(), yc.min(), yc.max()])*1000

logo_img = plt.imread('NERSC_LOGO_English-text.png')[:4450]
logo_img_white = logo_img.copy()[::5, ::5]
logo_img_white[logo_img_white[:,:,3] <1] = 1

In [ ]:
#for i_src_file in enumerate(src_files):
def plot_ice(i_src_file):
    i, src_file = i_src_file
    dst_png_file = f'{dst_dir}/frm_{prefix}_{i:04}.png'
    datestr = get_date(i, src_file)
    month = datestr[4:6]
    sia, _ = get_lmsia_data(src_file)
    #sia[np.isnan(sia)] = 0
    watermask = np.where(sia == 0, 1, np.nan)

    fig = plt.figure(figsize=figsize)
    ax = plt.axes((0,0,1,1), projection=plt_csr)
    imsh = ax.imshow(sia, extent=extent, clim=clim, cmap=cmap, zorder=1, interpolation='nearest')
    ax.imshow(watermask, extent=extent, clim=[0,1], cmap='gray_r', zorder=2, interpolation='nearest')
    ax.imshow(land, extent=extent, clim=[0, 2], cmap='gray', zorder=3, interpolation='nearest')
    ax.add_feature(cfeature.LAND, edgecolor='k', facecolor='none', zorder=3)
    ax.set_xlim((-2000000, 2200000))
    ax.set_ylim((-2400000, 2700000))
    
    ax.imshow(logo_img_white, extent=[-2000000, -1700000, 2400000, 2700000], zorder=4)#, interpolation='nearest')
    
    plt.text(-1800000, 2100000, f'{datestr[:4]}-{datestr[4:6]}-{datestr[6:]}', zorder=10, fontsize=16, color='k')
    ax1 = plt.axes((0.55,0.97,0.35,0.02))
    plt.colorbar(imsh, cax=ax1, orientation='horizontal', label='Average sea ice age, years', pad=0.05)
    legend_elements = [
        Patch(facecolor='black', edgecolor='black', label='water'),
        Patch(facecolor='white', edgecolor='black', label='no data')
    ]
    ax.legend(handles=legend_elements, loc='lower center', bbox_to_anchor=(0.9, 0.85), fontsize=12, frameon=False)
    plt.savefig(dst_png_file, bbox_inches='tight', pad_inches=0.05, facecolor='#000', dpi=dpi)
    #plt.show()
    plt.close('all')
    plt.close()
    gc.collect()
    
 

In [ ]:
with Pool() as pool:
    for _ in tqdm.tqdm(pool.imap(plot_ice, enumerate(src_files)), total=len(src_files)):
        pass

In [ ]:
!ffmpeg -y -r 15 -f image2 -i frm_ice_age_%04d.png -vcodec libx264 -vf 'crop=trunc(iw/2)*2:trunc(ih/2)*2:0:0' -pix_fmt yuv420p lm_ice_age.mp4
